# Classification with Delira and SciKit-Learn - A very short introduction
*Author: Justus Schock* 

*Date: 31.07.2019*

This Example shows how to set up a basic classification model and experiment using SciKit-Learn.

Let's first setup the essential hyperparameters. We will use `delira`'s `Parameters`-class for this:

In [ ]:
logger = None
from delira.training import Parameters
import sklearn
params = Parameters(fixed_params={
    "model": {},
    "training": {
        "batch_size": 64, # batchsize to use
        "num_epochs": 10, # number of epochs to train
        "optimizer_cls": None, # optimization algorithm to use
        "optimizer_params": {}, # initialization parameters for this algorithm
        "losses": {}, # the loss function
        "lr_sched_cls": None,  # the learning rate scheduling algorithm to use
        "lr_sched_params": {}, # the corresponding initialization parameters
        "metrics": {"mae": mean_absolute_error} # and some evaluation metrics
    }
}) 

c:\users\jsc7rng\appdata\local\conda\conda\envs\delira-dev\lib\site-packages\pywt\_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
c:\users\jsc7rng\appdata\local\conda\conda\envs\delira-dev\lib\site-packages\google\protobuf\descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message
c:\users\jsc7rng\appdata\local\conda\conda\envs\delira-dev\lib\site-packages\tensorflow\python\util\nest.py:1286: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Mapping", _collections.Mapping)
c:\users\jsc7rng\appdata\local\conda\conda\envs\delira-dev\lib\site-packages\tensorflow\p

Since we did not specify any metric, only the `CrossEntropyLoss` will be calculated for each batch. Since we have a classification task, this should be sufficient. We will train our network with a batchsize of 64 by using `Adam` as optimizer of choice.

## Logging and Visualization
To get a visualization of our results, we should monitor them somehow. For logging we will use `Tensorboard`. Per default the logging directory will be the same as our experiment directory.


## Data Preparation
### Loading
Next we will create some fake data. For this we use the `ClassificationFakeData`-Dataset, which is already implemented in `deliravision`. To avoid getting the exact same data from both datasets, we use a random offset.

In [ ]:
from deliravision.data.fakedata import ClassificationFakeData
dataset_train = ClassificationFakeData(num_samples=10000, 
                                       img_size=(3, 32, 32), 
                                       num_classes=10)
dataset_val = ClassificationFakeData(num_samples=1000, 
                                     img_size=(3, 32, 32), 
                                     num_classes=10,
                                     rng_offset=10001
                                     )

### Augmentation
For Data-Augmentation we will apply a few transformations:

In [ ]:
from batchgenerators.transforms import RandomCropTransform, \
                                        ContrastAugmentationTransform, Compose
from batchgenerators.transforms.spatial_transforms import ResizeTransform
from batchgenerators.transforms.sample_normalization_transforms import MeanStdNormalizationTransform

transforms = Compose([
    RandomCropTransform(24), # Perform Random Crops of Size 24 x 24 pixels
    ResizeTransform(32), # Resample these crops back to 32 x 32 pixels
    ContrastAugmentationTransform(), # randomly adjust contrast
    MeanStdNormalizationTransform(mean=[0.5], std=[0.5])]) 



With these transformations we can now wrap our datasets into datamanagers:

In [ ]:
from delira.data_loading import DataManager, SequentialSampler, RandomSampler

manager_train = DataManager(dataset_train, params.nested_get("batch_size"),
                                transforms=transforms,
                                sampler_cls=RandomSampler,
                                n_process_augmentation=4)

manager_val = DataManager(dataset_val, params.nested_get("batch_size"),
                              transforms=transforms,
                              sampler_cls=SequentialSampler,
                              n_process_augmentation=4)


## Model

After we have done that, we can specify our model: We will use a very simple MultiLayer Perceptron here. 
In opposite to other backends, we don't need to provide a custom implementation of our model, but we can simply use it as-is. It will be automatically wrapped by `SklearnEstimator`, which can be subclassed for more advanced usage.

## Training
Now that we have defined our network, we can finally specify our experiment and run it.

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning) # ignore UserWarnings raised by dependency code
warnings.simplefilter("ignore", FutureWarning) # ignore FutureWarnings raised by dependency code
from sklearn.neural_network import MLPClassifier

from delira.training import SklearnExperiment

if logger is not None:
    logger.info("Init Experiment")
experiment = PyTorchExperiment(params, MLPClassifier,
                               name="ClassificationExample",
                               save_path="./tmp/delira_Experiments",
                               key_mapping={"X": "X"}
                               gpu_ids=[0])
experiment.save()

model = experiment.run(manager_train, manager_val)

Congratulations, you have now trained your first Classification Model using `delira`, we will now predict a few samples from the testset to show, that the networks predictions are valid (for now, this is done manually, but we also have a `Predictor` class to automate stuff like this):

In [ ]:
import numpy as np
from tqdm.auto import tqdm # utility for progress bars

preds, labels = [], []

with torch.no_grad():
    for i in tqdm(range(len(dataset_val))):
        img = dataset_val[i]["data"] # get image from current batch
        img_tensor = img.astype(np.float) # create a tensor from image, push it to device and add batch dimension
        pred_tensor = model(img_tensor) # feed it through the network
        pred = pred_tensor.argmax(1).item() # get index with maximum class confidence
        label = np.asscalar(dataset_val[i]["label"]) # get label from batch
        if i % 1000 == 0:
            print("Prediction: %d \t label: %d" % (pred, label)) # print result
        preds.append(pred)
        labels.append(label)
        
# calculate accuracy
accuracy = (np.asarray(preds) == np.asarray(labels)).sum() / len(preds)
print("Accuracy: %.3f" % accuracy)